# Simulating genomic data in a 'plant breeding' context

In [1]:
using XSim, DelimitedFiles, Distributions

In [2]:
#using Pkg;Pkg.rm("XSim")
Pkg.update()

UndefVarError: UndefVarError: Pkg not defined

### Defining the genome and reading a real marker catalogue

In [118]:
#Pkg.add(PackageSpec(name="XSim",rev="master"))

In [3]:
mutRate    = 0.0
numChr     = 2
nLoci      = 6096
chrLength  = [1.62;1.41]
numLoci    = [3339;2757]
mutationRate = 0.0
myData = readdlm("markerCatalogue4JuliaChrom1-2", ' ', Any, '\n', header=false)
mp1 = Float64.(myData[1,1:numLoci[1]])
mp2 = Float64.(myData[2,1:numLoci[2]])
mapPos = [mp1, mp2]

genefreq1   = fill(0.5,numLoci[1])
genefreq2 = fill(0.5,numLoci[2])
geneFreq = [genefreq1, genefreq2]

idx = rand(numLoci[1]).>0.995  # you want 0.5% to be QTL, i.e about 17 QTL
qtlIndex1 = collect(1:numLoci[1])[idx]
idx = rand(numLoci[2]).>0.995  # you want 0.5%% to be QTL, i.e about 14 QTL
qtlIndex2 = collect(1:numLoci[2])[idx]
qtlIndex = [qtlIndex1, qtlIndex2]
line = 0
numQTL = 0

for i in qtlIndex
    line +=1
    println("Number of QTL on chromosome $line: ", length(i))
    numQTL += 1
end

qtlEffect1 = randn(length(qtlIndex1))/sqrt(0.5*numQTL)
qtlEffect2 = randn(length(qtlIndex2))/sqrt(0.5*numQTL)
qtlEffect = [qtlEffect1, qtlEffect2];

Number of QTL on chromosome 1: 15
Number of QTL on chromosome 2: 17


### Building the genome
Passing  arrays for chrLength, numLoci and arrays of arrays for mapPos, geneFreq, qtlIndex, qtlEffect
i.e. using the most flexible version of build_genome

In [4]:
build_genome(numChr,chrLength,numLoci,geneFreq, mapPos, qtlIndex, qtlEffect, mutationRate)

### Sampling founder individuals from a file with 1326 genotyped individuals
Note that the genotype data for two chromosomes on the file is phased. You may have to use FImpute or some similar software for phasing. You may read less individuals form the data file than it contains, but not more.

In [5]:
popSizeFounder = 1326
basePop = sampleFounders(popSizeFounder,"reformattedMarkerDataChrom1-2");

Sampling 1326 animals into base population.


MethodError: MethodError: no method matching zero(::Type{SubString{String}})
Closest candidates are:
  zero(!Matched::Type{LibGit2.GitHash}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/LibGit2/src/oid.jl:220
  zero(!Matched::Type{Pkg.Resolve.VersionWeights.VersionWeight}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Pkg/src/resolve/VersionWeights.jl:19
  zero(!Matched::Type{Pkg.Resolve.MaxSum.FieldValues.FieldValue}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Pkg/src/resolve/FieldValues.jl:44
  ...

Splitting the founder individuals into tow cohorts of equal size

In [122]:
basePopMales = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
basePopMales.animalCohort = basePop.animalCohort[1:663];

UndefVarError: UndefVarError: basePop not defined

In [ ]:
basePopFemales = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
basePopFemales.animalCohort = basePop.animalCohort[664:1326];

Although not used here, animals and cohorts belong to breeds, and the breed coposition can be set for cohorts and is inferered thereafter from the mating strategy

In [ ]:
XSim.setBreedComp(basePop,[1.0])
XSim.setBreedComp(basePopMales,[1.0])
XSim.setBreedComp(basePopFemales,[1.0]);

Removing output files form a previous run, then creating 5'000 DH lines from an initial crossing of the two base populations

In [ ]:
outputFileName = "JuliaPlants"
run(`\rm -f $outputFileName.ped`)
run(`\rm -f $outputFileName.phe`)
run(`\rm -f $outputFileName.brc`)
run(`\rm -f $outputFileName.gen`)

numOffspring = 5000
DHBaseLines1 = XSim.sampleDHLines(basePopMales, basePopFemales, numOffspring)
DHBaseLines2 = XSim.sampleDHLines(basePopMales, basePopFemales, numOffspring);

Lets look at the mean and variance of the genotypic values of our 5000 DH-lines in DHBaseline1 and DHBaseline2.  

In [ ]:
var(getOurGenVals(DHBaseLines1))

In [ ]:
var(getOurGenVals(DHBaseLines2))

In [ ]:
mean(getOurGenVals(DHBaseLines1))

In [ ]:
mean(getOurGenVals(DHBaseLines2))

Let us specify  the residual variance to simulate phenotypic values, note that getOurPhenVals() is actually setting all individuals phenotypic values in the cohort that is passed as an argument.

In [ ]:
resVar = 5
XSim.setResidualVariance(resVar)

In [ ]:
var(getOurPhenVals(DHBaseLines1,resVar))

In [ ]:
var(getOurPhenVals(DHBaseLines2,resVar))

In [ ]:
outputFileName

Now lets output the data to the files. Note that outputPedigree() is not only outputting the pedigree, but also the phenotypes and gentoypes, look at the files that got generated!

In [ ]:
outputPedigree(DHBaseLines1,outputFileName);
outputPedigree(DHBaseLines2,outputFileName);

Sampling each 2500 male and female parents from DHBaseline1 at random and producing one offspring each. Sampling of parents is done with replacement. Note that sampleRan() is sampling 50% female and 50% male offspring, we are concatenating them into a single cohort. Note further, that we are using the same population to sample male and female parents from, so sex is ignored and we might even end up cloning an individual. Furthermore, after this step of random mating, our DH-Lines are not double haploids anymore. With the parameter numGen that is passed as an argument to sampleRan(), we are determining the number of generations of random mating. The variable 'gen' is used to store the actual numer of generation the simualtion currently is at.

In [ ]:
DHLine1_H1 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine1_H1m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine1_H1f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numOffspring = 2500
numGen = 1
gen = 1
output = true
DHLine1_H1m, DHLine1_H1f, gen = sampleRan(numOffspring, numGen, DHBaseLines1, DHBaseLines1, fileName=outputFileName)
DHLine1_H1 = concatCohorts(DHLine1_H1m, DHLine1_H1f);

Now we are repeating the same random mating for DHBaseline2

In [ ]:
DHLine2_H1 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine2_H1m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine2_H1f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numOffspring = 2500
numGen = 1
gen = 1
output = true
DHLine2_H1m, DHLine2_H1f, gen = sampleRan(numOffspring, numGen, DHBaseLines2, DHBaseLines2, fileName=outputFileName)
DHLine2_H1 = concatCohorts(DHLine2_H1m, DHLine2_H1f);

We know now how to generate double haploid lines and how to do random mating. Next let's do phenotypic selection. As before we are declaring the male and female cohorts to hold the offspring that are going to be generated. We will be generation 1000 offspring based on 100 phenotypically best male and female parents. Again we will be selecting the phenotypically best 100 parents from the same population that was generated in the previous step (DHLine1_H1).

In [ ]:
DHLine1_H2 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine1_H2m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine1_H2f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numMaleParents = 100
numFemaleParents = 100
numOffspring = 1000
numGen = 1
DHLine1_H2m, DHLine1_H2f, gen = sampleSel(numOffspring, numMaleParents, numFemaleParents, numGen, 
                                    DHLine1_H1, DHLine1_H1, XSim.common.varRes, gen=gen,
                                    fileName=outputFileName, direction=1);
DHLine1_H2 = concatCohorts(DHLine1_H2m, DHLine1_H2f);

As before, we are applying mass selection also to the other population in the same way as was done above for DHLine_H1 now for DHLine_H2

In [ ]:
DHLine2_H2 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine2_H2m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
DHLine2_H2f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numMaleParents = 100
numFemaleParents = 100
numOffspring = 1000
numGen = 1
DHLine2_H2m, DHLine2_H2f, gen = sampleSel(numOffspring, numMaleParents, numFemaleParents, numGen, 
                                    DHLine2_H1, DHLine2_H1, XSim.common.varRes, gen=gen,
                                    fileName=outputFileName, direction=1);
DHLine2_H2 = concatCohorts(DHLine2_H2m, DHLine2_H2f);

After random mating and phentoypic selection, let us apply selection based on BLUP, i.e. breeding values. For this purpose, we need to read the pedigree and phenotpyic files that were produced sampleRan() and sampleSel() as well as outputPedigree() above. First, let's read the phenotypic data into a dataframe.

In [ ]:
using DataFrames
# BLUP selection
phenofile = outputFileName*".phe"
colNames = [:Animal;:y;:bv]
#dfPhen = CSV.read(phenofile,delim = ' ',header=false,names=colNames)
dfPhen = readtable(phenofile, separator=' ', header=false)
names!(dfPhen, colNames)
dfPhen[:1] = string.(dfPhen[:1])
dfPhen

Now, we read the pedigree in to a dataframe, reformat it, write is out again such that it can finally be read by the get_pedigree() method. This should be fixed in future releases....

In [ ]:
pedfile   = outputFileName*".ped"
dfPed = readtable(pedfile,separator = ' ', header = false)

In [ ]:
using CSV
pedfile = outputFileName*".pedReformatted"
CSV.write(pedfile, dfPed, delim=';',header=["ind","sire" , "dam"])

In [ ]:
ped = XSim.get_pedigree(pedfile,separator=';');

After having read phenotypes and genotypes, we can now set the genetic variance that shall be used in BLUP, define the model for analysis, setup the mixed model equations and solve them. 

In [ ]:
varGen = 5
mme = XSim.build_model("y = intercept + Animal",resVar)
XSim.set_random(mme,"Animal",ped,varGen)
out = XSim.solve(mme,dfPhen,solver="GaussSeidel",printout_frequency=40)

Now all our indiviudals have estimated genotypic values, with the method putEBV() we are transferring them to the indiviudals that we would like to select to become the parents of the next generation. In our case, these would be the cohorts DHLine1_H2 and DHLine2_H2

In [ ]:
# transfer BLUP-EBV to animals
XSim.putEBV(DHLine1_H2,ped,mme,out)
XSim.putEBV(DHLine2_H2,ped,mme,out)

In [ ]:
DHLine1_H2_sorted = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
DHLine2_H2_sorted  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
DHLine1_H3  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
DHLine2_H3  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
direction = 1.0
numMaleParents = 50
numFemaleParents = 50
numOffspring = 500
numGen = 1

After having declared empty cohorts for the offspring of the next generation, we are creating a y vector with the animals EBVs in the direction we want to select on (e.g. if low values arefavourable, direction should be -1.0). Then EBVs are sorted and the best number of parents are selected to be passed as parents to sampleChildren()

In [ ]:
println("Generation $gen : sampling children by BLUP")
y = direction*[animal.ebv for animal in DHLine1_H2.animalCohort]
DHLine1_H2_sorted.animalCohort = DHLine1_H2.animalCohort[sortperm(y)][(end-numMaleParents+1):end]
y = direction*[animal.ebv for animal in DHLine2_H2.animalCohort]
DHLine2_H2_sorted.animalCohort = DHLine2_H2.animalCohort[sortperm(y)][(end-numFemaleParents+1):end]
DHLine1_H3 = XSim.sampleChildren(DHLine1_H2_sorted,DHLine1_H2_sorted,numOffspring)
DHLine2_H3 = XSim.sampleChildren(DHLine2_H2_sorted,DHLine2_H2_sorted,numOffspring)
outputPedigree(DHLine1_H3,outputFileName)
outputPedigree(DHLine2_H3,outputFileName)
gen+=1

Finally, we are producing each 250 double haploid lines again, from the offspring of the BLUP selected parents 

In [ ]:
println("Producing DHLines again....")
DHLine1_4  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))
DHLine2_4  = XSim.Cohort(Array{XSim.Animal}(undef,0),Array{Int64}(undef,0,0))

numOffspring = 250
DHLine1_4 = XSim.sampleDHLines(DHLine1_H3, DHLine1_H3, numOffspring)
DHLine2_4 = XSim.sampleDHLines(DHLine2_H3, DHLine2_H3, numOffspring);
     


In [ ]:
outputPedigree(DHLine1_4,outputFileName);
outputPedigree(DHLine2_4,outputFileName);

The final step is crossing the two DHLines again to produce hybrids....

In [ ]:
println("Producing hybrids now....")
H1 = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
H1m = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0)) 
H1f = XSim.Cohort(Array{XSim.Animal,1}(undef,0),Array{Int64,2}(undef,0,0))
numOffspring = 250
numGen = 1
gen = 1
output = true
H1m, H1f, gen = sampleRan(numOffspring, numGen, DHLine1_4, DHLine2_4, fileName=outputFileName)
H1 = concatCohorts(DHLine2_H1m, DHLine2_H1f);


Of course the 'breeding program' outlined here is not reflecting a real situation but rahter shows you the tools XSim has to do simulations. You should be able to modify the above code to better reflect a real situation. Also you may modify XSim's methodsto better reflect the situation you need to cope with. Note that all data simulated is written to the files with the name <outputFileName> that you declared above. so You should be able to look at selection response, genetic trend across generation etc.